# Dashbord Waldbrände in Europa


> In diesem Notebook wird ein Dashboard mit Waldbranddaten aus Europa erstellt. Dafür werden im ersten Schritt die Grafiken für das Dashboard festgelegt und anschließend das Dashboard erzeugt. 
> Was nutze ich dafür? 

## Import und Transformieren der Daten

In [1]:
from pymongo import MongoClient
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data

In [2]:
client = MongoClient()
client = MongoClient('mongodb://localhost:27017/')
mydb = client["esa_fire"]
mycol = mydb["esa_fire_2001"]

In [3]:
df_countries = pd.read_csv('C:\dev\eda_esa_fire\doc\country_codes.csv', on_bad_lines='skip', delimiter=',')
europe = df_countries[df_countries['region'] == 'Europe']['alpha-2'].to_list()

In der CSV-Datei "country_codes.csv" sind Ländernamen inklusive verschiedenen Abkürzungen und Kontinenten enthalten. Mit Hilfe dieser Datei wird eine Liste mit zweistelligen Ländercodes aus Europa erstellt. Später wird der Dataframe ebenfalls genutzt um die vollständigen Namen zu erhalten. 

In [4]:
list_europe = list(mycol.find({"country":{"$in":europe}}))
df_eu = pd.DataFrame(list_europe)
df_eu.head(5)

,_id,lon,lat,country,date,burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches,no_data_burned_area,tree_cover_broadleaved_evergreen_burned_area,tree_cover_broadleaved_deciduous_burned_area,tree_cover_needleleaved_evergreen_burned_area,tree_cover_needleleaved_deciduous_burned_area,tree_cover_mixed_leave_burned_area,cropland_burned_area,mosaic_tree_grass_burned_area,other_vegetation_burned_area,flooded_area_burned_area
0,630630b328dd00e55a25462f,-3.625,56.125,GB,2001-01-01,804970.0,545465.0,0.829803,0.959910,3.0,751305.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53664.0,0.0
1,630630b328dd00e55a254630,-3.625,55.875,GB,2001-01-01,2039257.0,707836.0,0.938498,0.977980,1.0,1502610.0,0.0,0.0,0.0,0.0,0.0,375652.0,0.0,160994.0,0.0
2,630630b328dd00e55a254631,22.875,53.125,PL,2001-01-01,3005221.0,838988.0,0.996411,0.981294,1.0,0.0,0.0,590311.0,0.0,0.0,0.0,0.0,0.0,53664.0,2361245.0
3,630630b328dd00e55a254632,24.625,51.875,UA,2001-01-01,1609940.0,720975.0,0.974907,0.963874,3.0,965964.0,0.0,429317.0,0.0,0.0,0.0,0.0,0.0,214658.0,0.0
4,630630b328dd00e55a254633,26.125,51.875,UA,2001-01-01,21948848.0,929124.0,0.991455,0.826491,2.0,1287952.0,0.0,268323.0,0.0,0.0,0.0,697640.0,0.0,1985592.0,17709340.0


Mit der Query `"country":{"$in":europe}}` wird ein Dataframe mit alle Waldbranddaten aus Europa erzeugt. 

In [5]:
df_eu['year'] = df_eu['date'].str.slice(start=0, stop=4)

Zur besseren Visualisierung wird eine Jahr-Spalte ergänzt. 

## Festlegung der Grafiken 

In [16]:
# TODO Grafiken beschreiben 

In [6]:
df_test = df_eu[(df_eu['year'] == '2016') & (df_eu['country'] == 'IT')]

### Area Plot: Summe der verbrannten Fläche pro Monat

In [7]:
def create_area_plot(df):
    df = df.groupby(by= 'date').sum().reset_index()
    fig = px.area(df, x="date", y="burned_area")
    fig.update_traces(line_color='#FD7207')
    fig.update_layout(title = 'Burend Area per Month', plot_bgcolor= "rgb(250, 250, 250)",paper_bgcolor="rgb(255, 255, 255)")
    return fig

In [8]:
fig_1 = create_area_plot(df_test)
fig_1.show()

### Treemap: Summe der verprannten Fläche pro Vegetationsklasse

In [9]:
def create_tree_map(df):
    df = df.groupby(by= 'year').sum().reset_index()
    df= df.drop(columns= ['burned_area', 'year', 'lon', 'lat', 'standard_error', 'fraction_of_burnable_area', 'fraction_of_observed_area', 'number_of_patches'] )
    df = df.transpose().reset_index()
    df = df[(df[0] != 0)]
    fig = px.treemap(df, 
        path = [px.Constant('All'), 'index'], 
        color = 0, 
        color_continuous_scale='Emrld',
        values = 0)
    fig.update_layout(title = 'Burend Area per Vegetation Class')
    return fig

In [10]:
fig_2 = create_tree_map(df_test)
fig_2.show()

### Geo-Scatter-Plot: Lage der verbrannten Fläche 

In [11]:
def create_scatter_geo(df):
    fig = go.Figure(data=go.Scattergeo(
         lon=df["lon"], lat= df["lat"], 
         mode = 'markers',
         marker = dict(
            size = 6,
            reversescale = True,
            symbol = 'square',
            line = dict(
                width=0.5,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'Hot',
            cmin = 0,
            color = df['burned_area'],
            cmax = df['burned_area'].max(),
            colorbar_title="Burend Area"
        ))
    )
    fig.update_layout(
        title = 'Burned Area',
        geo = dict(
            scope='europe',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
        ),)
    return fig

In [12]:
fig_3 = create_scatter_geo(df_test)
fig_3.show()

## Dashboard

Im Dashboard sollen die oben erstellten Grafiken dargestellt werden. Über zwei Drop-Down-Menüs sollen die Grafiken an Länder und Jahreszahlen angepasst werden. Im ersten Schritt müssen für die Menüs die Werte und Labels gerneriert werden. 

In [13]:
df_labels = df_countries[df_countries['region'] == 'Europe'][['name', 'alpha-2']]
df_labels.rename(columns={'name': 'label', 'alpha-2': 'value'}, inplace = True)
option_countries= df_labels.to_dict('records')

Das Drop-Down-Menü der Länder wird über den oben eingelesen Länder-Dataframe erzeugt, wobei die Labels die vollständigen Länderbezeichnungen sind und die Values, die im Datensatz verwendet Kurzbezeichnungen. 

In [14]:
unique_dates  = df_eu['year'].unique()
option_dates =  list()
for data in unique_dates: 
    date_dict = {'label': data, 'value': data}
    option_dates.append(date_dict)

*Values* und *Lables* des zweiten Drop-Down-Menüs sind die Jahresbezeichnungen aus dem Datensatz. 

In [15]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Wild Fires Europe", style = {
                    'color': '#3C5C79',
                    'font-family': '"Open Sans", verdana, arial, sans-serif'
                }),
    html.Label([
        "Country",
    dcc.Dropdown(id="selected_country",
                 options= option_countries,
                 multi=False,
                 style={'width': "40%"}
                 )],style = {
                    'color': '#3C5C79',
                    'font-family': '"Open Sans", verdana, arial, sans-serif'
                }),
    html.Label([ "Years",
    dcc.Dropdown(id= "selected_year",
                 options= option_dates,
                 multi=False,
                 style={'width': "40%"}
                 ), ], style = {
                    'color': '#3C5C79',
                    'font-family': '"Open Sans", verdana, arial, sans-serif'
                }), 
    html.Div([ 
            html.Div([
                dcc.Graph(id = 'fig_1'),
            ], className='three columns'),
            html.Div([
                dcc.Graph(id = 'fig_2' ),
            ], className='three columns'),
            html.Div([
                dcc.Graph(id = 'fig_3' ),
            ], className='three columns'),
        ], className = 'row'),
    ])


@app.callback(
    [Output('fig_1' , 'figure'),
     Output('fig_2' , 'figure'), 
     Output('fig_3' , 'figure')],
    Input("selected_country", "value"), 
    Input("selected_year", "value")
)

def update_figure(selected_country, selected_year):

    if not selected_country:
        selected_country = 'IT'
        selected_year = '2016'
    dff = df_eu.copy()
    dff = dff[dff["country"] == selected_country]
    dff = dff[dff["year"] == selected_year]

    fig_1 = create_scatter_geo(dff)
    fig_2 = create_area_plot(dff)
    fig_3 = create_tree_map(dff)
    return [fig_1, fig_2, fig_3]
app.run_server(mode='inline')

Verherende Waldbrandjahr von 2010-2020 in Deutschland waren vor allem 2015 und 2018 (Quelle). Filter man nach Deutschland und 2018 ermöglicht das Dashboard einen Überlick über die Situation. Interessant in dem Jahr ist die Aufteilung der Brände nach Vegetationsklassen. Auffällig ist die große Zahl an verbrannter Fläche bei *flooded_area*. Ein Ursache dafür kann der verheerend [Moorbrand in Meppen 2018](https://www.ndr.de/nachrichten/niedersachsen/osnabrueck_emsland/Moorbrand-Landkreis-Emsland-uebt-Kritik-an-Bundeswehr,moorbrand1172.html) gewesen sein, welcher sich im September ereignet hat. Meppen lässt sich als Waldbrandgebiet auf der Karte wiederfinden. Im  Flächendiagram ist ebenfalls im September ein Pike. 